In [1]:
## 直接读取原始数据，并且保留原来的文本和图像，并把tag和title拼接
import random
import os
import re
import json
import numpy as np
import pandas as pd
import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import torch
import lightgbm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
random_seed = 2020
random.seed(random_seed)
np.random.seed(random_seed)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# all_data = pd.read_csv('../data/feature_data_530.csv')
all_data = pd.read_csv('../data/forauto.csv')
# train_all_data = all_data[:-180581]
# submit_all_data = all_data[-180581:]
# all_data
# glove
glove_tags = pd.read_csv('../data/alltags_feature.csv')
glove_title = pd.read_csv('../data/title_feature.csv')
vilt_flickr = pd.read_csv('../data/vilt_fea_flickr.csv')
all_data = pd.concat([all_data, glove_tags, glove_title,vilt_flickr], axis=1)

# feature_columns = ['img_file']
# feature_columns = ['Pid', 'train_type', 'mean_label','img_file'] 
# feature_columns += ['user_fe_{}'.format(i) for i in range(399)]
# feature_columns += ['loc_fe_{}'.format(i) for i in range(400)]
all_data = all_data.drop(feature_columns, axis=1)
train_all_data = all_data[:-180581]
submit_all_data = all_data[-180581:]
all_data

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,title_fe_290,title_fe_291,title_fe_292,title_fe_293,title_fe_294,title_fe_295,title_fe_296,title_fe_297,title_fe_298,title_fe_299
0,21894,4,5,65,75,33,6,128,12,333,...,-0.055784,0.197610,-0.166620,-0.007064,0.308736,0.138135,0.149598,0.184107,-0.049617,0.011684
1,53866,13,0,75,139,56,13,800,65,500,...,0.296695,0.087271,0.126468,-0.103056,0.147320,0.193802,0.035001,-0.045986,0.080715,0.078193
2,26948,1,3,42,480,12,2,188,23,500,...,0.168270,0.003270,0.171345,0.108478,0.349160,-0.396890,-0.205695,0.068778,-0.128545,-0.080683
3,355,1,10,68,225,18,3,61,9,500,...,0.109710,0.117009,0.245160,-0.306995,0.387836,0.111045,0.157754,-0.086720,0.220020,-0.045470
4,315,31,2,43,317,8,1,146,19,500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,2,0,35,301,6,1,78,13,500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486190,12280,7,0,60,110,25,4,164,23,500,...,0.134555,0.186670,0.330580,0.227405,0.155600,-0.088244,0.144885,-0.199950,0.033038,-0.065841
486191,43496,9,4,28,186,41,7,85,9,500,...,0.156818,-0.107772,0.157098,-0.079490,0.042594,-0.034590,-0.349488,0.016242,-0.172004,-0.012735
486192,5492,17,7,27,54,15,3,305,35,500,...,0.142362,-0.097873,-0.026855,0.195713,-0.101540,0.240027,-0.119417,-0.012023,0.021850,-0.081305


In [29]:
label = 'label'
from autogluon.tabular import FeatureMetadata
from autogluon.tabular import TabularDataset, TabularPredictor
feature_metadata = FeatureMetadata.from_df(train_all_data)

print(feature_metadata)

('float', [])        : 646 | ['svd_mode_t_0', 'svd_mode_t_1', 'svd_mode_t_2', 'svd_mode_t_3', 'svd_mode_t_4', ...]
('int', [])          :  37 | ['Uid', 'Uid_count', 'Category', 'Subcategory', 'Concept', ...]
('object', [])       :   3 | ['Category.1', 'Concept.1', 'Subcategory.1']
('object', ['text']) :   2 | ['Alltags', 'Title']


In [30]:
excluded_model_types = ['GBM','CAT','VW','NN_TORCH']
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config
hyperparameters = get_hyperparameter_config('multimodal')

hyperparameters

{'NN_TORCH': {},
 'GBM': [{},
  {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
  'GBMLarge'],
 'CAT': {},
 'XGB': {},
 'AG_AUTOMM': {},
 'VW': {}}

In [31]:
from autogluon.tabular import TabularPredictor
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4'
predictor = TabularPredictor(label=label).fit(
    train_data=train_all_data,
    hyperparameters=hyperparameters,
    excluded_model_types=excluded_model_types,
    feature_metadata=feature_metadata,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    # num_bag_folds=5, 
    ag_args_fit={'num_gpus': 2}
    # hyperparameters = {'NN_TORCH': {'num_epochs': 5}, 'GBM': {'num_boost_round': 20}},
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230527_063009/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230527_063009/"
AutoGluon Version:  0.7.0
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #78~20.04.1-Ubuntu SMP Wed Apr 19 11:26:48 UTC 2023
Train Data Rows:    305613
Train Data Columns: 687
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (16.56, 1.0, 6.40552, 2.47301)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may 

In [24]:
leaderboard = predictor.leaderboard(submit_all_data)

Load pretrained checkpoint: /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230526_111730/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/model.ckpt
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Load pretrained checkpoint: /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230526_111730/models/MultiModalPredictor_BAG_L1/S1F2/automm_model/model.ckpt
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, 

                        model  score_test  score_val  pred_time_test  pred_time_val      fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         VowpalWabbit_BAG_L1   -6.474182  -2.170498     3142.466837    1201.360886   3773.821155              3142.466837             1201.360886        3773.821155            1       True          4
1             CatBoost_BAG_L1   -6.505793  -1.015767        1.341930       0.663028    424.088711                 1.341930                0.663028         424.088711            1       True          1
2  MultiModalPredictor_BAG_L1   -6.516147  -1.341388      181.519297      65.557616   9144.728142               181.519297               65.557616        9144.728142            1       True          5
3       NeuralNetTorch_BAG_L1   -6.527236  -1.112051       28.855850      10.308606    762.461131                28.855850               10.308606         762.461131            1       True       

In [32]:
all_models = predictor.get_model_names()
all_models

['CatBoost_BAG_L1',
 'XGBoost_BAG_L1',
 'NeuralNetTorch_BAG_L1',
 'VowpalWabbit_BAG_L1',
 'MultiModalPredictor_BAG_L1',
 'WeightedEnsemble_L2',
 'CatBoost_BAG_L2',
 'XGBoost_BAG_L2',
 'NeuralNetTorch_BAG_L2',
 'WeightedEnsemble_L3']

In [33]:
i = 0  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred0 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred0.iloc[0]))

Prediction from CatBoost_BAG_L1 model: 9.485945


In [34]:
i = 1  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred1 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred1.iloc[0]))

/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)


Prediction from XGBoost_BAG_L1 model: 9.514761


In [35]:
i = 2  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred2 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred2.iloc[0]))

/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dis

Prediction from NeuralNetTorch_BAG_L1 model: 8.669583


In [36]:
i = 3  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred3 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred3.iloc[0]))

Prediction from VowpalWabbit_BAG_L1 model: 8.654921


In [37]:
i = 4  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred4 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred4.iloc[0]))

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text/config.json
Model config ElectraConfig {
  "_name_or_path": "/MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation"

Prediction from MultiModalPredictor_BAG_L1 model: 9.721411


In [38]:
i = 5  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred5 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred5.iloc[0]))

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text/config.json
Model config ElectraConfig {
  "_name_or_path": "/MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation"

Prediction from WeightedEnsemble_L2 model: 9.525716


In [39]:
i = 6  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred6 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred6.iloc[0]))

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text/config.json
Model config ElectraConfig {
  "_name_or_path": "/MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation"

Prediction from CatBoost_BAG_L2 model: 9.651156


In [40]:
i = 7  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred7 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred7.iloc[0]))

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text/config.json
Model config ElectraConfig {
  "_name_or_path": "/MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation"

Prediction from XGBoost_BAG_L2 model: 9.550249


In [41]:
i = 8  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred8 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred8.iloc[0]))

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text/config.json
Model config ElectraConfig {
  "_name_or_path": "/MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation"

Prediction from NeuralNetTorch_BAG_L2 model: 9.559865


In [43]:
i = 9  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred9 = predictor.predict(submit_all_data.drop(columns=[label]), model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred9.iloc[0]))

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text/config.json
Model config ElectraConfig {
  "_name_or_path": "/MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230527_063009/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/hf_text",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation"

Prediction from WeightedEnsemble_L3 model: 9.568596


In [44]:
model_pred0.to_csv('../data/auto_5f_CAT1.csv',header=True, index=None)
model_pred1.to_csv('../data/auto_5f_XGB1.csv',header=True, index=None)
model_pred2.to_csv('../data/auto_5f_NNT1.csv',header=True, index=None)
model_pred3.to_csv('../data/auto_5f_VW1.csv',header=True, index=None)
model_pred4.to_csv('../data/auto_5f_MMP1.csv',header=True, index=None)
model_pred5.to_csv('../data/auto_5f_Ensemble2.csv',header=True, index=None)
model_pred6.to_csv('../data/auto_5f_CAT2.csv',header=True, index=None)
model_pred7.to_csv('../data/auto_5f_XGB2.csv',header=True, index=None)
model_pred8.to_csv('../data/auto_5f_NNT2.csv',header=True, index=None)
model_pred9.to_csv('../data/auto_5f_Ensemble3.csv',header=True, index=None)


In [45]:
leaderboard = predictor.leaderboard(submit_all_data)

/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/pandas/core/arrays/base.py:513: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(self, dtype=dtype)
/usr/local/lib/python3.8/dis

                        model  score_test  score_val  pred_time_test  pred_time_val       fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0             CatBoost_BAG_L1   -6.224059  -0.966294       12.424019      12.136761    1886.828896                12.424019               12.136761        1886.828896            1       True          1
1         VowpalWabbit_BAG_L1   -6.246320  -1.513337     3073.269047    1009.125686    4108.309258              3073.269047             1009.125686        4108.309258            1       True          4
2       NeuralNetTorch_BAG_L1   -6.270536  -1.085602       34.347759      15.062787    1576.037658                34.347759               15.062787        1576.037658            1       True          3
3             CatBoost_BAG_L2   -6.337632  -0.859033     4371.714132    1687.604743  140676.680133                11.849617               14.120359         240.653024            2       True  

In [ ]:
# 'CatBoost_BAG_L1',
#  'XGBoost_BAG_L1',
#  'NeuralNetTorch_BAG_L1',
#  'VowpalWabbit_BAG_L1',
#  'MultiModalPredictor_BAG_L1',
#  'WeightedEnsemble_L2',
#  'CatBoost_BAG_L2',
#  'XGBoost_BAG_L2',
#  'NeuralNetTorch_BAG_L2',
#  'WeightedEnsemble_L3'

In [25]:
y_pred = predictor.predict(submit_all_data.drop(columns=[label]))
y_pred.head()

Load pretrained checkpoint: /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230526_111730/models/MultiModalPredictor_BAG_L1/S1F1/automm_model/model.ckpt
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Load pretrained checkpoint: /MMChallenge/SMP/HyFea-main/AutogluonModels/ag-20230526_111730/models/MultiModalPredictor_BAG_L1/S1F2/automm_model/model.ckpt
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, 

305613     8.664865
305614    10.856478
305615     9.701542
305616     6.663115
305617     6.663115
Name: label, dtype: float32

In [26]:
y_pred.to_csv('../data/auto_5fold_5models.csv',header=True, index=None)

In [14]:
y_pred = predictor.predict(submit_all_data.drop(columns=[label]))
y_pred.head()

305613     8.541639
305614    10.987025
305615    10.023292
305616     6.948594
305617     6.948594
Name: label, dtype: float32

In [15]:
np.mean(y_pred)

6.304041

In [16]:
y_pred.to_csv('../data/auto_4models.csv',header=True, index=None)